# Кредитный скоринг

При принятии решения о выдаче кредита или займа учитывается т.н. «Кредитный скоринг» — рейтинг платежеспособности клиента. ИИ на основе модели, которую просчитывает машинное обучение — в ней много параметров — возраст, зарплата, кредитная история, наличие недвижимости, автомобиля, судимости и других признаков, после обработки которых выносится положительное или отрицательное решение

In [1]:
# Импортируем библиотеки
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Данные:

**application_record.csv**
*   Feature name	Explanation	Remarks
*   ID	Client number	
*   CODE_GENDER	Gender	
*   FLAG_OWN_CAR	Is there a car	
*   FLAG_OWN_REALTY	Is there a property	
*   CNT_CHILDREN	Number of children	
*   AMT_INCOME_TOTAL	Annual income	
*   NAME_INCOME_TYPE	Income category	
*   NAME_EDUCATION_TYPE	Education level	
*   NAME_FAMILY_STATUS	Marital status	
*   NAME_HOUSING_TYPE	Way of living	
*   DAYS_BIRTH	Birthday	Count backwards from current day (0), -1 means yesterday
*   DAYS_EMPLOYED	Start date of employment	Count backwards from current day(0). If positive, it means the person currently unemployed.
FLAG_MOBIL	Is there a mobile phone	
*   FLAG_WORK_PHONE	Is there a work phone	
*   FLAG_PHONE	Is there a phone	
*   FLAG_EMAIL	Is there an email	
*   OCCUPATION_TYPE	Occupation	
*   CNT_FAM_MEMBERS	Family size	

**credit_record.csv**
*   Feature name	Explanation	Remarks
*   ID	Client number	
*   MONTHS_BALANCE	Record month	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on
*   STATUS	Status	
   *   0: 1-29 days past due
   *   1: 30-59 days past due 
   *   2: 60-89 days overdue 
   *   3: 90-119 days overdue 
   *   4: 120-149 days overdue 
    *   5: Overdue or bad debts, write-offs for more than 150 days
    *   C: paid off that month X: No loan for the month


## Считываем данные

In [2]:
# Ниже, мы для тех, у кого хоть раз были просрчоки больше 60 дней, ставим в таргет 1.
data = pd.read_csv("application_record.csv", encoding = 'utf-8')
record = pd.read_csv("credit_record.csv", encoding = 'utf-8')

In [3]:
# Добавляем срок кредита к параметрам выдачи кредита
# минимальный баланс за месяц
begin_month = pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min) * - 1)
begin_month = begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'}) 
new_data = pd.merge(data, begin_month, how="left",on="ID") 

In [4]:
# Больше 60, то это просрочка, ставим - Yes, 
# если просрочка есть за срок кредита,то так же ставим Yes
record['dep_value'] = None
record['dep_value'][record['STATUS'] == '2'] = 'Yes'
record['dep_value'][record['STATUS'] == '3'] = 'Yes'
record['dep_value'][record['STATUS'] == '4'] = 'Yes'
record['dep_value'][record['STATUS'] == '5'] = 'Yes'
cpunt = record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0] = 'Yes' 
cpunt['dep_value'][cpunt['dep_value'] == 0] = 'No'

In [5]:
cpunt

,MONTHS_BALANCE,STATUS,dep_value
ID,,,
5001711,4,4,No
5001712,19,19,No
5001713,22,22,No
5001714,15,15,No
5001715,60,60,No
...,...,...,...
5150482,18,18,No
5150483,18,18,No
5150484,13,13,No


In [6]:
# # Джойним всё вместе,заменяем Yes и No на 1 и 0
cpunt = cpunt[['dep_value']]
new_data = pd.merge(new_data, cpunt, how='inner', on='ID')
new_data['target'] = new_data['dep_value']
new_data.loc[new_data['target'] == 'Yes','target'] = 1
new_data.loc[new_data['target'] == 'No','target'] = 0

In [7]:
#  В итоге к анкетным данным мы добавили таргет
new_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value,target
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,15.0,No,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,14.0,No,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,-1134,1,0,0,0,Security staff,2.0,29.0,No,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,4.0,No,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,26.0,No,0


In [8]:
# Упростим себе задачу и оставим только часть признаков
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']
target = ['target',]
dataset = new_data[features + target]
dataset[target[0]] = pd.to_numeric(dataset[target[0]])

In [9]:
new_data

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value,target
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,15.0,No,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,-4542,1,1,0,0,NaN,2.0,14.0,No,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,-1134,1,0,0,0,Security staff,2.0,29.0,No,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,4.0,No,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,-3051,1,0,1,1,Sales staff,1.0,26.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,...,-2420,1,0,0,0,Managers,2.0,11.0,Yes,1
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,...,-1325,1,0,1,1,Medicine staff,2.0,23.0,Yes,1
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,...,-1325,1,0,1,1,Medicine staff,2.0,32.0,Yes,1
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,...,-655,1,0,0,0,Sales staff,2.0,9.0,Yes,1


In [10]:
# Разделим выборку на трейн и тест, на трейн будем обучать модель, на тест валидировать.
X_train, X_test, y_train, y_test = train_test_split(dataset[features], pd.to_numeric(dataset[target[0]]), test_size=0.3, random_state=42)

In [11]:
# Превращаем категориальные факторы в численные
ohe = OneHotEncoder()
ohe.fit(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
X_train_ohe = ohe.transform(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
X_test_ohe = ohe.transform(X_test[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])

X_train_ohe = pd.DataFrame(X_train_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])
X_test_ohe = pd.DataFrame(X_test_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])

In [12]:
# Отскалируем численные
mms = MinMaxScaler()
mms.fit(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
X_train_scaled = mms.transform(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
X_test_scaled = mms.transform(X_test[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])

X_train_scaled = pd.DataFrame(X_train_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])

In [13]:
X_train = pd.concat([X_train_scaled, X_train_ohe,], axis=1)
X_test = pd.concat([X_test_scaled, X_test_ohe, ], axis=1)

#  Модель

In [14]:
# Создадим простейшую модель, которая покажет через линейные коэффиценты связь переменных и таргета
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [15]:
train_score, test_score = accuracy_score(model.predict(X_train), y_train), accuracy_score(model.predict(X_test), y_test)
print(f'Точность модели на трейне {train_score}, на тесте {test_score}')

Точность модели на трейне 0.9828755045260394, на тесте 0.983635033827025


In [16]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import keyword_only

from pyspark.ml import Transformer, Estimator, Pipeline
from pyspark.ml.param.shared import (HasInputCols, 
                                     HasInputCol, 
                                     HasOutputCol, 
                                     Param, 
                                     Params, 
                                     TypeConverters
                                    )
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml.feature import (Tokenizer, 
                                CountVectorizer, 
                                IDF, 
                                VectorAssembler, 
                                ElementwiseProduct,
                                StringIndexer,
                                MinMaxScaler
                               )
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector, SparseVector
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [17]:
spark = SparkSession.builder \
    .master('local[*]')\
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
    .appName("Spark") \
    .getOrCreate()

sc = spark.sparkContext

23/01/30 08:58:23 WARN Utils: Your hostname, ubuntu-d resolves to a loopback address: 127.0.1.1; using 192.168.0.101 instead (on interface enp7s0)
23/01/30 08:58:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/30 08:58:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
spark

### - Написать TfIdf на спарке (потом сделать это же со SparkMl)

In [19]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36457 entries, 0 to 36456
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   36457 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  int64  
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  int64  
 11  DAYS_EMPLOYED        36457 non-null  int64  
 12  FLAG_MOBIL           36457 non-null  int64  
 13  FLAG_WORK_PHONE      36457 non-null  int64  
 14  FLAG_PHONE           36457 non-null  int64  
 15  FLAG_EMAIL           36457 non-null 

In [20]:
# чтобы было, на чём строить вектор:
text_feat = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 
             'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
df_text = spark.createDataFrame(new_data).withColumn('text', F.concat_ws(', ', *text_feat))
df_text.select('text').show(3, truncate=False);

+----------------------------------------------------------------------------------+
|text                                                                              |
+----------------------------------------------------------------------------------+
|Working, Higher education, Civil marriage, Rented apartment                       |
|Working, Higher education, Civil marriage, Rented apartment                       |
|Working, Secondary / secondary special, Married, House / apartment, Security staff|
+----------------------------------------------------------------------------------+
only showing top 3 rows



In [21]:
# Tokenizer
class MyTokenizer(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    
    @keyword_only
    def __init__(self, inputCol: str, outputCol: str):
        super(MyTokenizer, self).__init__()
        kwargs = self._input_kwargs
        self._set(**kwargs)
    
    def _transform(self, df: pyspark.sql.DataFrame):
        return df.withColumn(self.getOutputCol(), F.split(F.lower(self.getInputCol()), '\W+'))

In [22]:
myt = MyTokenizer(inputCol='text', outputCol='t_text')
data_t_text = myt.transform(df_text)
data_t_text.select('text', 't_text').show(3, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------
 text   | Working, Higher education, Civil marriage, Rented apartment                          
 t_text | [working, higher, education, civil, marriage, rented, apartment]                     
-RECORD 1--------------------------------------------------------------------------------------
 text   | Working, Higher education, Civil marriage, Rented apartment                          
 t_text | [working, higher, education, civil, marriage, rented, apartment]                     
-RECORD 2--------------------------------------------------------------------------------------
 text   | Working, Secondary / secondary special, Married, House / apartment, Security staff   
 t_text | [working, secondary, secondary, special, married, house, apartment, security, staff] 
only showing top 3 rows



In [23]:
# дальше - запуталась с передачей параметров + индексы для RDD
# class TFTransf(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
#     dictionary = Param(Params._dummy(), "dictionary", "dictionary", typeConverter=None)
    
#     @keyword_only
#     def __init__(self, inputCol: str, outputCol: str, dictionary = None):
#         super(TFTransf, self).__init__()
#         if dictionary is None:
#             raise ValueError("no dictionary")
#         kwargs = self._input_kwargs
#         self._set(**kwargs)
    
#     def _transform(self, df: pyspark.sql.DataFrame):
# #         tmp_col = df.select(self.getInputCol()).rdd.map(lambda x: Vectors.dense(getTF(x, dictionary)), VectorUDT())
#         tmp_col = df.select('t_text').rdd.map(lambda x: (x[0], getTF(x[1], self.dictionary)))
# #         tmp_df = spark.createDataFrame(tmp_col,
# #                                StructType([
# #                                    StructField("ID", LongType()),
# #                                    StructField("TF", ArrayType(FloatType()))
# #                                ]))
#         return df.join(tmp_col, on='ID', how='left')

In [24]:
# class TFEst(Estimator, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    
#     @keyword_only
#     def __init__(self, inputCol: str, outputCol: str):
#         super(TFEst, self).__init__()
#         kwargs = self._input_kwargs
#         self._set(**kwargs)
        
#     def _fit(self, df: pyspark.sql.DataFrame):
#         # список уникальных токенов [t1, t2, t3...]
#         dictionary = (df.select(self.getInputCol())
#                         .rdd
#                         .flatMap(lambda row: row[0])
#                         .distinct().collect()
#                      )   
#         return TFTransf(inputCol=self.getInputCol(), outputCol=self.getOutputCol(), dictionary=dictionary)

In [25]:
# tfe = TFEst(inputCol='t_text', outputCol='TF')
# tfe.fit(data_t_text).transform(data_t_text)

In [26]:
# поэтому просто без обёртки
# словарь для подсчёта частот - fit
data_rdd = data_t_text.select('t_text').rdd
my_dict = data_rdd.flatMap(lambda row: row[0]).distinct().collect()
my_dict[:3]

['rented', 'staff', 'separated']

In [27]:
# + столбец для TF - трансформация + нужен индекс
# DenseVector - тк нагляднее
def getTF(row, dictionary):
    row = row
    tmp_vect = [0 for i in range(len(dictionary))]
    for t in row:
        for i in range(len(dictionary)):
            if dictionary[i] == t:
                tmp_vect[i] += 1
                continue
#     tmp_vect = Vectors.dense([x / len(row) for x in tmp_vect])
    tmp_vect = [x / len(row) for x in tmp_vect]
    return tmp_vect

In [28]:
# + ID для Join
tmp_col = data_t_text.select('ID', 't_text').rdd.map(lambda x: (x[0], getTF(x[1], my_dict)))
tmp_col.take(1)

[(5008804,
  [0.14285714285714285,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.14285714285714285,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.14285714285714285,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.14285714285714285,
   0.0,
   0.0,
   0.0,
   0.14285714285714285,
   0.0,
   0.0,
   0.14285714285714285,
   0.14285714285714285,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0])]

In [29]:
# rdd to df
tmp_df = spark.createDataFrame(tmp_col,
                               StructType([
                                   StructField("ID", LongType()),
#                                    StructField("TF", VectorUDT())
                                   StructField("TF", ArrayType(DoubleType()))
                               ]))
tmp_df.show(1, truncate=50)                          

+-------+--------------------------------------------------+
|     ID|                                                TF|
+-------+--------------------------------------------------+
|5008804|[0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
+-------+--------------------------------------------------+
only showing top 1 row



In [30]:
# собственно transform: data + TF
(data_t_text.join(tmp_df, on='ID', how='left')
            .select('ID', 't_text', 'TF')
            .show(3, truncate=40)
)

+-------+----------------------------------------+----------------------------------------+
|     ID|                                  t_text|                                      TF|
+-------+----------------------------------------+----------------------------------------+
|5008804|[working, higher, education, civil, m...|[0.14285714285714285, 0.0, 0.0, 0.0, ...|
|5008805|[working, higher, education, civil, m...|[0.14285714285714285, 0.0, 0.0, 0.0, ...|
|5022611|[working, secondary, secondary, speci...|[0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0,...|
+-------+----------------------------------------+----------------------------------------+
only showing top 3 rows



In [31]:
data_tf = data_t_text.join(tmp_df, on='ID', how='left')

In [32]:
data_tf.printSchema()

root
 |-- ID: long (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: long (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: long (nullable = true)
 |-- DAYS_EMPLOYED: long (nullable = true)
 |-- FLAG_MOBIL: long (nullable = true)
 |-- FLAG_WORK_PHONE: long (nullable = true)
 |-- FLAG_PHONE: long (nullable = true)
 |-- FLAG_EMAIL: long (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)
 |-- begin_month: double (nullable = true)
 |-- dep_value: string (nullable = true)
 |-- target: long (nullable = true)
 |-- text: string (nullable = false)
 |-- t_text: array (nullable = false)
 |  

In [33]:
dict_size = len(my_dict)
collect_size = data_tf.count()
dict_size, collect_size

(55, 36457)

In [34]:
# сложение векторов
idf_counter = data_tf.select('TF').rdd.map(lambda vec: [1 if x > 0 else 0 for x in vec[0]]).reduce(lambda x, y: [x[i] + y[i] for i in range(len(x))])
idf_counter[:5]

[575, 12127, 2103, 174, 174]

In [35]:
idf_coeff = [np.log(collect_size/idf_counter[i]) for i in range(dict_size)]
idf_coeff[:5]

[4.149518722097133,
 1.1006991122376917,
 2.8527685871917834,
 5.344833463679954,
 5.344833463679954]

In [36]:
# idf_coeff = sc.broadcast(str(idf_coeff))

In [37]:
# далее - (для разнообразия не rdd) умножить, чтобы получить вектор и это всё ужасно криво, да
@F.pandas_udf(ArrayType(DoubleType()))
def getIDF(col, idf):
    import numpy as np 
    idf = eval(idf.values[0])
    return col.apply(lambda x: np.multiply(x, idf))

In [38]:
data_tf.select(getIDF(F.col('TF'), F.lit(str(idf_coeff))).alias('TFIDF')).show(truncate=75)

+---------------------------------------------------------------------------+
|                                                                      TFIDF|
+---------------------------------------------------------------------------+
|[0.5927883888710189, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...|
|[0.5927883888710189, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...|
|[0.0, 0.09172492601980764, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
|[0.0, 0.0, 0.47546143119863055, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
|[0.0, 0.0, 0.47546143119863055, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
|[0.0, 0.0, 0.0, 0.0, 0.0, 0.19355837662441736, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
|[0.0, 0.0, 0.0, 0.0, 0.0, 0.19355837662441736, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
|[0.0, 0.0, 0.0, 0.0, 0.0, 0.19355837662441736, 0.0, 0.0, 0.0, 0.0, 0.0, ...|
|[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.094466640...|
|[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

**TF-IDF со SparkML**

In [39]:
tokenizer = Tokenizer(inputCol='text', outputCol='tokens')
cv = CountVectorizer(inputCol=tokenizer.getOutputCol(), outputCol='vectors')
idf = IDF(inputCol=cv.getOutputCol(), outputCol='IDF')

In [40]:
pipe = Pipeline(stages=[
    tokenizer,
    cv,
    idf
])

In [41]:
pipe.fit(df_text).transform(df_text).select('vectors', 'IDF').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                           vectors|                                               IDF|
+--------------------------------------------------+--------------------------------------------------+
|(58,[6,8,9,10,22,23,39],[1.0,1.0,1.0,1.0,1.0,1....|(58,[6,8,9,10,22,23,39],[0.6612407789534962,1.2...|
|(58,[6,8,9,10,22,23,39],[1.0,1.0,1.0,1.0,1.0,1....|(58,[6,8,9,10,22,23,39],[0.6612407789534962,1.2...|
|(58,[0,1,2,3,4,5,6,7,38],[2.0,2.0,1.0,1.0,1.0,1...|(58,[0,1,2,3,4,5,6,7,38],[0.06353632823959426,0...|
|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[3.0,2.0,1.0...|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[0.095304492...|
|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[3.0,2.0,1.0...|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[0.095304492...|
|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[3.0,2.0,1.0...|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[0.095304492...|
|(58,[0,1,2,3,4,5,7,11,12,15,16,18],[3.0,2.0,1.0...|(58,[0,1,2,3

### - Обучить свою линейную регрессию

- тк задача на классификацию, линейную регрессию поняла как логистическую регрессию

- считывание данных и добавление таргета

In [42]:
data = spark.read.csv('application_record.csv', inferSchema = True, header = True, encoding = 'utf-8')
record = spark.read.csv('credit_record.csv', inferSchema = True, header = True, encoding = 'utf-8')

In [43]:
data.columns, record.columns

(['ID',
  'CODE_GENDER',
  'FLAG_OWN_CAR',
  'FLAG_OWN_REALTY',
  'CNT_CHILDREN',
  'AMT_INCOME_TOTAL',
  'NAME_INCOME_TYPE',
  'NAME_EDUCATION_TYPE',
  'NAME_FAMILY_STATUS',
  'NAME_HOUSING_TYPE',
  'DAYS_BIRTH',
  'DAYS_EMPLOYED',
  'FLAG_MOBIL',
  'FLAG_WORK_PHONE',
  'FLAG_PHONE',
  'FLAG_EMAIL',
  'OCCUPATION_TYPE',
  'CNT_FAM_MEMBERS'],
 ['ID', 'MONTHS_BALANCE', 'STATUS'])

In [44]:
# минимальный баланс за месяц
begin_month = record.groupBy('ID').min('MONTHS_BALANCE').select('ID', F.col('min(MONTHS_BALANCE)') * -1).withColumnRenamed('(min(MONTHS_BALANCE) * -1)', 'begin_month')
begin_month.show(3)

+-------+-----------+
|     ID|begin_month|
+-------+-----------+
|5001812|         22|
|5001849|          8|
|5001921|         19|
+-------+-----------+
only showing top 3 rows



In [45]:
new_data = data.join(begin_month, on='ID', how='left')
new_data.columns

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'begin_month']

In [46]:
record.select('STATUS').distinct().show()

+------+
|STATUS|
+------+
|     3|
|     0|
|     5|
|     C|
|     X|
|     1|
|     4|
|     2|
+------+



In [47]:
# таргет
condition = F.when((F.col('STATUS') == '2') | (F.col('STATUS') == '3') | (F.col('STATUS') == '4') | (F.col('STATUS') == '5'), 1).otherwise(0)
record_t = record.withColumn('target', condition)
record_t.show(3)

+-------+--------------+------+------+
|     ID|MONTHS_BALANCE|STATUS|target|
+-------+--------------+------+------+
|5001711|             0|     X|     0|
|5001711|            -1|     0|     0|
|5001711|            -2|     0|     0|
+-------+--------------+------+------+
only showing top 3 rows



In [48]:
# число совпадает
record_t.select(F.sum(F.col('target'))).show()

+-----------+
|sum(target)|
+-----------+
|       3104|
+-----------+



In [49]:
cpunt = record_t.groupby('ID').sum('target').select('ID', F.when(F.col('sum(target)') > 0, 1).otherwise(0).alias('target'))
# кол-во класса 1 также совпадает - 667
cpunt.show(3)

+-------+------+
|     ID|target|
+-------+------+
|5001812|     0|
|5001849|     0|
|5001921|     0|
+-------+------+
only showing top 3 rows



In [50]:
new_data = new_data.join(cpunt, on='ID', how='left').na.fill(0, 'target')

In [51]:
new_data.select(F.sum('target')).show()

+-----------+
|sum(target)|
+-----------+
|        616|
+-----------+



In [52]:
new_data.columns

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'begin_month',
 'target']

- предобработка и обучение модели

In [53]:
# новый текстовый признак
text_feat = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 
             'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
new_data = new_data.withColumn('text', F.concat_ws(', ', *text_feat))
df_text.select('text').show(3, truncate=False);

+----------------------------------------------------------------------------------+
|text                                                                              |
+----------------------------------------------------------------------------------+
|Working, Higher education, Civil marriage, Rented apartment                       |
|Working, Higher education, Civil marriage, Rented apartment                       |
|Working, Secondary / secondary special, Married, House / apartment, Security staff|
+----------------------------------------------------------------------------------+
only showing top 3 rows



In [54]:
cat_features = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']  # по 2 уникальных => 1 столбец для каждого
num_features = ['AMT_INCOME_TOTAL', 'CNT_CHILDREN']

dataset = new_data.select(*cat_features, *num_features, 'text', 'target')
dataset.columns

['CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'AMT_INCOME_TOTAL',
 'CNT_CHILDREN',
 'text',
 'target']

In [55]:
# трейн и тест
X_train, X_test = dataset.randomSplit([0.7, 0.3])

In [56]:
# препроцессинг
se = StringIndexer(inputCols=cat_features, outputCols=[col + '_se' for col in cat_features])
tokenizer = Tokenizer(inputCol='text', outputCol='tokens')
cv = CountVectorizer(inputCol='tokens', outputCol='vectors')
idf = IDF(inputCol='vectors', outputCol='IDF')
va = VectorAssembler(inputCols=se.getOutputCols() + num_features + ['IDF'], outputCol='features')
mms = MinMaxScaler(inputCol='features', outputCol='scaled_feats')

In [57]:
stages = [se, tokenizer, cv, idf, va, mms]
pipe = Pipeline(stages=stages)

In [58]:
scaled_dataset = pipe.fit(X_train).transform(X_train)

In [59]:
scaled_dataset.printSchema()

root
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- text: string (nullable = false)
 |-- target: integer (nullable = true)
 |-- CODE_GENDER_se: double (nullable = false)
 |-- FLAG_OWN_CAR_se: double (nullable = false)
 |-- FLAG_OWN_REALTY_se: double (nullable = false)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectors: vector (nullable = true)
 |-- IDF: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_feats: vector (nullable = true)



In [60]:
# тюнинг модели
lr = LogisticRegression(featuresCol='scaled_feats', labelCol='target')

# lr_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', 
#                                              labelCol='target', 
#                                              metricName='areaUnderROC')

m_evaluator = MulticlassClassificationEvaluator(labelCol="target", 
                                                predictionCol="prediction", 
                                                metricName='accuracy')

params = (ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.5, 0.9, 1.2])
                            .addGrid(lr.elasticNetParam, [0.2, 0.5, 0.7])
                            .build()
              )

tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=params,
                           evaluator=m_evaluator,
#                            evaluator=lr_evaluator
                          )

In [61]:
# лучшая accuracy
lr_model = tvs.fit(scaled_dataset)
max(lr_model.validationMetrics)

0.9985614897735

In [63]:
# получаем предсказание
fitted_pipe = pipe.fit(X_train)
train_pred = lr_model.transform(fitted_pipe.transform(X_train))
test_pred = lr_model.transform(fitted_pipe.transform(X_test))

In [65]:
# получаем метрики
m_evaluator.evaluate(train_pred), m_evaluator.evaluate(test_pred)

(0.9985716702673156, 0.9986509660455013)

Метрики чуть лучше, чем со sklearn (на трейне 0.9828755045260394, на тесте 0.983635033827025)

In [66]:
spark.stop()